In [1]:
%pip install setuptools==70.3.0  # apparently setuptools < 71.0.0 is required for a required package of openml to install on here
%pip install openml
%pip install numpy
%pip install pandas
# cf. https://pytorch.org/get-started/locally/
#%pip install torch torchvision torchaudio  # cuda 12.4, cf. nvidia-smi shell command
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu  # cpu version
%pip install -U scikit-learn
%pip install scipy
%pip install -U pymoo
%pip list

Note: you may need to restart the kernel to use updated packages.


Das System kann die angegebene Datei nicht finden.


Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: '#': Expected package name at the start of dependency specifier
    #
    ^


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Package              Version
-------------------- -----------
about-time           4.2.1
alive-progress       3.2.0
argon2-cffi          23.1.0
argon2-cffi-bindings 21.2.0
asttokens            3.0.0
autograd             1.7.0
certifi              2025.1.31
cffi                 1.17.1
charset-normalizer   3.4.1
cma                  3.2.2
colorama             0.4.6
comm                 0.2.2
contourpy            1.3.1
cramjam              2.9.1
cycler               0.12.1
deap                 1.4.2
debugpy              1.8.12
decorator            5.1.1
Deprecated           1.2.18
dill                 0.3.9
executing            2.2.0
fastparquet          2024.11.0
filelock             3.17.0
fonttools            4.56.0
fsspec               2025.2.0
grapheme             0.6.0
idna               

In [2]:
from openml import tasks

from classes import EAGGA

import os

In [3]:
oml_task_ids = [37, 43, 3903, 3904, 3913, 3918, 10093, 9946, 146819, 359955, 189922, 359962, 190392, 167120, 190137, 190410, 168350, 359975, 359972, 146820]
oml_tasks = tasks.get_tasks(oml_task_ids)

oml_datasets = [oml_task.get_dataset() for oml_task in oml_tasks]

# define positive classes
positive_classes = ['tested_positive', '1', True, True, 'yes', True, '2', '2', '1', '2', '1', True, '1', '1', '2', '1', '2', 'Anomaly', '1', '2']

f:\workplace_github\ws2425-tab-ml\code\.venv\Lib\site-packages\openml\tasks\functions.py:372: UserWarning: `download_data` will default to False starting in 0.16. Please set `download_data` explicitly to suppress this warning.
  warnings.warn(
f:\workplace_github\ws2425-tab-ml\code\.venv\Lib\site-packages\openml\tasks\functions.py:380: UserWarning: `download_qualities` will default to False starting in 0.16. Please set `download_qualities` explicitly to suppress this warning.
  warnings.warn(


In [4]:
hps = {
    'total_layers': (3, 10),
    'nodes_per_hidden_layer': (3, 20),
    'mu': 15,  # TODO: 100
    'lambda': 5,  # TODO: 10
    'holdout_train_size': 2/3,
    'cv_k': 5
}

batch_size = 64
patience = 10

secs_per_fold = 30
secs_total = 20 * 60

for (oml_dataset, class_positive) in zip(oml_datasets[:1], positive_classes[:1]):  # TODO: remove [:1]
    name = oml_dataset.name
    print(f'Dataset {name}')

    file_path = os.path.join('export', name)
    
    eagga = EAGGA(
        oml_dataset=oml_dataset,
        class_positive=class_positive,
        hps=hps,
        batch_size=batch_size,
        patience=patience,
        secs_per_fold=secs_per_fold,
        secs_total=secs_total,
        file_path=file_path
    )
    #eagga.load_population(6)
    eagga.run_eagga()

Dataset diabetes
Starting init population
Finished init population
Start EAGGA at 2025-03-02T23:16:51.014690
Generation 1, evaluate 5 individuals
Running 5-fold CV for individual: 4 total layers, 3 nodes per hidden layer, gs: ([0, 2, 4, 6, 7], [[[1, 3, 5], 0]])
Stop early: 0.6568940788507461 < 0.6568993628025055
Fold 1/5 | trained for 48 epochs | stopped early: True | metrics: {'loss': 0.6401613354682922, 'auc': np.float64(0.4937810945273632), 'nf': 0.375, 'ni': 0.10714285714285714, 'nnm': 0.375}
Stop early: 0.6463037222623825 < 0.6465589106082916
Fold 2/5 | trained for 195 epochs | stopped early: True | metrics: {'loss': 0.6715280413627625, 'auc': np.float64(0.4112769485903814), 'nf': 0.375, 'ni': 0.10714285714285714, 'nnm': 0.375}
Stop early: 0.6864931672811508 < 0.6866236329078674
Fold 3/5 | trained for 55 epochs | stopped early: True | metrics: {'loss': 0.7076503038406372, 'auc': np.float64(0.2298507462686567), 'nf': 0.375, 'ni': 0.10714285714285714, 'nnm': 0.375}
Stop early: 0.635

KeyboardInterrupt: 